# Fit ellipses

## Imports

In [ ]:
# Python standard library
import sys
import csv
import os

# Scientific computing

import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage
import sklearn.mixture

#plt.rc('text', usetex=True)
plt.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]

# Program specific
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi/')
import optical_imaging as oi
import oi_file
import image_processing


# Jupyter
from IPython.display import HTML

## Load data

In [ ]:
date = '10-10-2017'
particle = '293-T_0'
channel = '10-20-10_1'
file_index = '0'

base_path = oi_file.data_base_path

oi_vid_file_path = base_path + date + '/'+ particle + '/' + channel + '/oi/bin/test_camera_' + file_index
oi_events_file_path = base_path + date + '/' + particle + '/' + channel + '/oi/events/test_camera_' + file_index + '_events' + '.json'

In [ ]:
# Load video
res_x = 384
res_y = 112
oi_fps = 250000
exposure = 500*10**(-9.)
oi_vid = oi_file.Video(oi_vid_file_path, res_x, res_y, oi_fps, exposure)

# Load events
oi_events = oi_file.open_event_file_json(oi_events_file_path)

print 'loaded', len(oi_events), 'oi events'

# Create stage

In [ ]:
template_index = 500

template_frame = oi_vid.get_frame(template_index)

plt.imshow(template_frame, cmap = 'gray', origin = 'lower')
plt.show()

In [ ]:
reload(oi)
template_frame = oi_vid.get_frame(template_index)

stage_file_path = base_path + date + '/' + particle + '/' + channel + '/oi/stage/stage_' + file_index
print stage_file_path
cs = oi.load_stage_file(stage_file_path)
print cs
c0 = cs[0]
c1 = cs[1]
c2 = cs[2]
c3 = cs[3]

oi_stage = oi.Stage(template_frame, c0, c1, c2, c3)
oi_stage.plot_stage()

## Define plotting functions

In [ ]:
def plot_frame(frame, show = True):
    plt.imshow(frame, cmap = 'gray', origin = 'lower', interpolation = 'none')
    
    if show:
        plt.show()
    
def plot_processed_over_raw(processed_frame, frame, show = True):
    green_processed_frame = np.zeros((processed_frame.shape[0], processed_frame.shape[1], 3))
    green_processed_frame[:,:,1] = processed_frame

    plt.imshow(frame, cmap = 'gray', origin = 'lower', alpha = 1, interpolation = 'none')
    plt.imshow(green_processed_frame, cmap = 'gray', origin = 'lower', alpha = 0.15, interpolation = 'none')

    if show:
        plt.show()
    
def plot_frame_hist(frame, show = True):
    plt.hist(frame.flatten(), facecolor = 'k', bins = 100)
    
    if show:
        plt.show()
    
def plot_all(processed_frame, frame):
    fig, axes = plt.subplots(1,3,figsize = (12,3))
    
    # Plot 1
    plt.sca(axes[0])
    plot_frame(processed_frame, show = False)
    
    
    # Plot 2
    plt.sca(axes[1])
    plot_processed_over_raw(processed_frame, frame, show = False)
    
    # Plot 3
    plt.sca(axes[2])
    plot_frame_hist(processed_frame, show = False)
    
    plt.show()

In [ ]:
def process_frame(frame, template_frame, detection, debug = False):
    
    processed_frame = np.copy(frame)
    processed_template_frame = np.copy(template_frame)
    
    # Crop
    function = crop_step['function']
    kwargs = crop_step['kwargs']            
    kwargs['x'] = detection._px
    kwargs['y'] = detection._py
    processed_frame = function(processed_frame, **kwargs)
    processed_template_frame = function(processed_template_frame, **kwargs)
    cropped_frame = np.copy(processed_frame)
    
    if debug:
        print function.__name__, kwargs
        #plot_all(processed_frame, cropped_frame)
    
    
    # Pre-negative steps
    for prenegative_step in prenegative_steps:
        function = prenegative_step['function']
        kwargs = prenegative_step['kwargs']
        
        processed_frame = function(processed_frame, **kwargs)
        processed_template_frame = function(processed_template_frame, **kwargs)
        
        if debug:
            print function.__name__, kwargs
            plot_all(processed_frame, cropped_frame)
        
    # Negative step
    function = negative_step['function']
    kwargs = negative_step['kwargs']
    processed_frame = function(processed_frame, processed_template_frame, **kwargs)
    
    if debug:
        print function.__name__, kwargs
        plot_all(processed_frame, cropped_frame)
        
    # Post-negative steps
    for postnegative_step in postnegative_steps:
        function = postnegative_step['function']
        kwargs = postnegative_step['kwargs']
        
        processed_frame = function(processed_frame, **kwargs)
        
        if debug:
            print function.__name__, kwargs
            plot_all(processed_frame, cropped_frame)
            
    return processed_frame
  
def fit_plot_ellipse(frame, processed_frame, crop_distance, detection, detection_index, debug = False):

    # Ellipse-fitting
    
    
    ellipse_pixels = np.where(processed_frame == 1)
        
    ellipse = oi.fit_ellipse_image_aligned(ellipse_pixels[1], ellipse_pixels[0])
    
    
    # Get ellipse parameters
    ellipse_x, ellipse_y = oi.get_ellipse_center(ellipse)
    ellipse_a, ellipse_b = oi.get_ellipse_axes_lengths(ellipse)
    ellipse_theta = oi.get_ellipse_angle(ellipse)
    
    cropped_frame = image_processing.crop_frame(frame, detection._px, detection._py, crop_distance)
    
    
    # Plot
    
    if debug:
        fig, axes = plt.subplots(1,3,figsize = (12,6))

        green_processed_frame = np.zeros((processed_frame.shape[0], processed_frame.shape[1], 3))
        green_processed_frame[:,:,1] = np.copy(processed_frame)
        




        fig.sca(axes[0])
        plt.imshow(cropped_frame, cmap = 'gray', origin = 'lower', interpolation = 'none')
        
        plt.xticks([])
        plt.yticks([])


        fig.sca(axes[1])
        plt.imshow(cropped_frame, interpolation = 'none', origin = 'lower', alpha = 1., cmap = 'gray')
        plt.imshow(green_processed_frame, interpolation = 'none', origin = 'lower', alpha = 0.5)
        
        plt.xticks([])
        plt.yticks([])

        fig.sca(axes[2])
        plt.imshow(cropped_frame, interpolation = 'none', origin = 'lower', alpha = 1., cmap = 'gray')
        plt.imshow(green_processed_frame, interpolation = 'none', origin = 'lower', alpha = 0.5)
        
        plt.xticks([])
        plt.yticks([])
        
        
        num_points = 100
        xs = []
        ys = []
        for i in range(num_points):
            angle = i*2*np.pi/99.
            temp_x = ellipse_a*np.cos(angle)
            temp_y = ellipse_b*np.sin(angle)
            xs.append(ellipse_x + np.cos(ellipse_theta)*temp_x + np.sin(ellipse_theta)*temp_y)
            ys.append(ellipse_y + np.sin(ellipse_theta)*temp_x - np.cos(ellipse_theta)*temp_y)
            
            
        plt.plot(xs, ys, ls = '--', c = 'white', lw = 1)
        
        plt.text(0, 1.0, 'a/b=' + str(round(ellipse_a/ellipse_b, 2)), transform = plt.gca().transAxes,\
                 color = 'white', fontweight = 'bold', size = 24, ha = 'left', va = 'top')
        
        plt.xlim(0, cropped_frame.shape[1] - 1)
        plt.ylim(0, cropped_frame.shape[0] - 1)
            
            


        plt.show()
    
    
    
    
    ellipse_x = ellipse_x + detection._px - crop_distance
    ellipse_y = ellipse_y + detection._py - crop_distance
    
    
    return [detection_index, ellipse_x, ellipse_y, ellipse_a, ellipse_b, ellipse_theta]
        
    
        

## Find event to test fit on

##### Frame

In [ ]:
fig, axes = plt.subplots(6,6,figsize = (10,5))
start_index = 36*6

for i in range(36):
    oi_index = start_index + i
    oi_event = oi_events[oi_index]
    ts = oi_event.get_tf()
    t = int((ts[-1] + ts[0])/2.)
    frame = oi_vid.get_frame(t)
    
    
    row = i/6
    column = i%6
    plt.sca(axes[row, column])
    plt.imshow(frame, cmap = 'gray', origin = 'lower')
    plt.title('event #' + str(i + start_index))
    plt.xticks([])
    plt.yticks([])
plt.show()

##### Video

In [ ]:
oi_index = 248
oi_event = oi_events[oi_index]
reload(oi_file)
plt.close()

#oi_file.make_animation(oi_vid, oi_event._detections[0]._tf, oi_event._detections[-1]._tf, oi_fps).save('test.mp4')
HTML(oi_file.make_animation(oi_vid, oi_event._detections[0]._tf, oi_event._detections[-1]._tf, oi_fps).to_html5_video())

## Define ellipse fitting functions

In [ ]:
#reload(image_processing)
oi_index = 241
oi_event = oi_events[oi_index]

In [ ]:
xc_left = 0
xc_right = 150
for j, detection in enumerate(oi_event._detections):
    try:
        x = int(detection._px)
        y = int(detection._py)
        xc, yc = oi_stage.get_channel_coordinates(x, y)
        xc = oi_stage.pixels_to_meters(xc)

        if xc >= xc_left and xc <= xc_right:

            tf = detection._tf

            frame = oi_vid.get_frame(tf)
            template_frame = oi_vid.get_frame(template_index)

            crop_distance = 30
            crop_step = {'function': image_processing.crop_frame, 'kwargs': {'crop_distance': crop_distance}}

            prenegative_steps = []
            prenegative_steps.append({'function': image_processing.normalize, 'kwargs': {}})
            prenegative_steps.append({'function': image_processing.gaussian_blur, 'kwargs': {'blur_kernel': (15, 15)}})


            negative_step = {'function': image_processing.negative, 'kwargs': {'direction':'neg'}}


            postnegative_steps = []
            postnegative_steps.append({'function': image_processing.normalize, 'kwargs': {}})
            postnegative_steps.append({'function': image_processing.gradient, 'kwargs': {}})
            postnegative_steps.append({'function': image_processing.invert, 'kwargs': {}})
            postnegative_steps.append({'function': image_processing.gaussian_threshold, 'kwargs': {'sigma_multiplier':0.5}})
            postnegative_steps.append({'function': image_processing.largest_cluster, 'kwargs': {}})
            #postnegative_steps.append({'function': image_processing.threshold_clusters, 'kwargs': {'cluster_threshold': 20}})
            postnegative_steps.append({'function': image_processing.morphological_closing, 'kwargs': {'morph_kernel': (45,45)}})
            postnegative_steps.append({'function': image_processing.erodesubtract, 'kwargs': {'iterations':3}})

            processed_frame = process_frame(frame, template_frame, detection, debug = False)
            fit_plot_ellipse(frame, processed_frame, crop_distance, detection, j, debug = True)



        
    except:
        print 'failed on ', j, len(oi_event._detections)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        pass
    
        

## Save fit

In [ ]:
def out_of_bounds(detection):
    x_min = 20
    x_max = res_x - 20
    
    y_min = 20
    y_max = res_y - 20
    
    
    out = False
    if detection._px < x_min:
        out = True
        
    if detection._px > x_max:
        out = True
        
    if detection._py < y_min:
        out = True
        
    if detection._py > y_max:
        out = True
    
    
    return out

In [ ]:
ellipsess = []
for i, oi_event in enumerate(oi_events):
    print 'i = ', i, '/', len(oi_events)

    ellipsess.append([])
    ellipses = ellipsess[-1]
    
    
    
    for j, detection in enumerate(oi_event._detections):
        try:

            
            if out_of_bounds(detection) == False:


                
                tf = detection._tf

                frame = oi_vid.get_frame(tf)
                template_frame = oi_vid.get_frame(template_index)


                processed_frame = process_frame(frame, template_frame, detection, debug = False)
                ellipse = fit_plot_ellipse(frame, processed_frame, crop_distance, detection, j, debug = False)

        
                ellipses.append(ellipse)
        
        
        
        except:
            #print 'failed on ', j, len(oi_event._detections)
            exc_type, exc_obj, exc_tb = sys.exc_info()
            fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
            #print(exc_type, fname, exc_tb.tb_lineno)
            pass



## Save

In [ ]:
output_base_path = oi_file.ellipse_base_path
output_file_path = output_base_path + date + '/' + particle + '/' + channel + '/' + file_index + '/' + 'ellipses'
print output_file_path

In [ ]:
preprocessing_steps = []

for step in [crop_step] + prenegative_steps + [negative_step] + postnegative_steps:
    preprocessing_steps.append([step['function'].__name__, step['kwargs']])
    
print preprocessing_steps

In [ ]:
with open(output_file_path, 'w') as file_handle:
    file_writer = csv.writer(file_handle, delimiter = '\t')
    
    # Write preprocessing steps
    file_writer.writerow(preprocessing_steps)
    
    # Write ellipse info
    file_writer.writerow(['detection index', 'ellipse center x', 'ellipse center y', 'ellipse axes x', 'ellipse axes y', 'ellipse angle'])
    
    # Write ellipse
    for i, ellipses in enumerate(ellipsess):
        file_writer.writerow(['event #', i])
        for ellipse in ellipses:
            file_writer.writerow(ellipse)